In [10]:
import pandas as pd
import numpy as np

In [26]:
# load and shift data so score corrisponds to next day
data_combined = pd.read_csv('./data/Combined_News_DJIA.csv')
data_combined.Label = data_combined.Label.shift(1)
data_combined = data_combined[1:]
stories = [c for c in data_combined if c.startswith('Top')]
data_combined = pd.melt(data_combined, id_vars=['Date','Label'], value_vars=stories, value_name='Headline')
#data_combined = data_combined.drop(['variable'], axis=1)
data_combined['Headline'] = data_combined['Headline'].apply(lambda x: str(x))
data_combined.head()

,Date,Label,variable,Headline
0,2008-08-11,0.0,Top1,b'Why wont America and Nato help us? If they w...
1,2008-08-12,1.0,Top1,b'Remember that adorable 9-year-old who sang a...
2,2008-08-13,0.0,Top1,b' U.S. refuses Israel weapons to attack Iran:...
3,2008-08-14,0.0,Top1,b'All the experts admit that we should legalis...
4,2008-08-15,1.0,Top1,"b""Mom of missing gay man: Too bad he's not a 2..."


In [31]:
# randomize
random_data = data_combined.sample(frac=1)

# split data
TT_SPLIT = .8
index = int(TT_SPLIT*len(data_combined))
train = random_data[:index]
test = random_data[index:]

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(sublinear_tf=True, 
                        min_df=100, norm='l2', 
                        encoding='latin-1', 
                        ngram_range=(1, 4), 
                        stop_words='english')

features = tfidf.fit_transform(train.Headline).toarray()
labels = data_combined.Label

logistic = LogisticRegression()
logistic = logistic.fit(features, train["Label"])

test_vectors = tfidf.transform(test.Headline)
predictions = logistic.predict(test_vectors)

print(classification_report(test['Label'], preds2))
print(accuracy_score(test['Label'], predictions))

             precision    recall  f1-score   support

        0.0       0.48      0.31      0.38      4582
        1.0       0.55      0.71      0.62      5358

avg / total       0.51      0.52      0.51      9940

0.5240442655935613


In [46]:
average_acc = 0

for _ in range(10):

    # randomize
    random_data = data_combined.sample(frac=1)

    # split data
    TT_SPLIT = .8
    index = int(TT_SPLIT*len(data_combined))
    train = random_data[:index]
    test = random_data[index:]

    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=100, norm='l2', encoding='latin-1', 
                            ngram_range=(1, 4), stop_words='english')

    features = tfidf.fit_transform(train.Headline).toarray()
    labels = data_combined.Label

    logistic = LogisticRegression()
    logistic = logistic.fit(features, train["Label"])

    test_vectors = tfidf.transform(test.Headline)
    predictions = logistic.predict(test_vectors)

    #print(classification_report(test['Label'], preds2))
    print('Run',_,':',accuracy_score(test['Label'], predictions))
    average_acc += accuracy_score(test['Label'], predictions)
print()
print('Average Accuracy:', average_acc/10)

Run 0 : 0.5105633802816901
Run 1 : 0.5080482897384306
Run 2 : 0.5187122736418511
Run 3 : 0.5099597585513078
Run 4 : 0.5142857142857142
Run 5 : 0.5172032193158954
Run 6 : 0.5127766599597585
Run 7 : 0.5259557344064386
Run 8 : 0.5214285714285715
Run 9 : 0.5045271629778671

Average Accuracy: 0.5143460764587525
